# naver 주식 종목 토론방 크롤링 

### 필요한 모듈 import

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import requests
from urllib.request import urlopen
import lxml.html
import time
import json
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import csv

In [2]:
base_url = 'https://finance.naver.com/item/board.nhn?code='

### 크롤링 함수

In [3]:
def naver_crawler(acode, page):
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3)'}
    total_dt = pd.DataFrame([])
    
    n_ = 0
    for page in range(1, page):
        
        n_ += 1
        if (n_ % 10 == 0):
            print('================== Page ' + str(page) + ' is done ==================')

        # url = "https://finance.naver.com/item/board.nhn?code=%s&page=%s" % (acode, page)
        url = "https://finance.naver.com/item/board.nhn?code=%s&page=%s" % (str(acode), str(page))

        result = requests.get(url, headers = headers)
        bs_obj = BeautifulSoup(result.content, "html.parser")
        table = bs_obj.find('table', {'class' : 'type2'})
        tt = table.select('tbody > tr')

        for i in range(2, len(tt)):
            if len(tt[i].select('td > span')) > 0:
              # print(i)
              date = tt[i].select('td > span')[0].text
              title = tt[i].select('td.title > a')[0]['title']
              writer = tt[i].select('td.p11')[0].text.replace('\t', '').replace('\n', '')
              views = tt[i].select('td > span')[1].text
              pos = tt[i].select('td > strong')[0].text
              neg = tt[i].select('td > strong')[1].text
              # print(date, title, writer, views, pos, neg)
              
              table = pd.DataFrame({'날짜' : [date], 
                                    '제목' : [title],
                                    '글쓴이' : [writer],
                                    '조회' : [views],
                                    '공감' : [pos],
                                    '비공감' : [neg]
                                    })
              total_dt = total_dt.append(table)
    
    return total_dt

### 주식 종목별 코드

In [4]:
acode={"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
      "ecopro":"086520","shinsungeng":"011930","unison":"018000",
      "kakao":"035720","naver":"035420","nc_s":"036570",
      "sds":"018260","samsung_e":"005930","sk_h":"000660",
      "skc":"011790","hansol":"014680","DB":"000990",
      "lotte_shoping":"023530","shinse":"004170","samsungcnt":"028260",
      "dongsuh":"026960","kb":"105560","korea":"071050","shinhan":"055550",
      "woori":"316140","hana":"086790"}

In [5]:
for k,v in acode.items():
    print(k,v)

호텔신라 008770
두산퓨얼셀 336260
한화솔루션 009830
ecopro 086520
shinsungeng 011930
unison 018000
kakao 035720
naver 035420
nc_s 036570
sds 018260
samsung_e 005930
sk_h 000660
skc 011790
hansol 014680
DB 000990
lotte_shoping 023530
shinse 004170
samsungcnt 028260
dongsuh 026960
kb 105560
korea 071050
shinhan 055550
woori 316140
hana 086790


In [6]:
for k,v in acode.items():
    print(k)

호텔신라
두산퓨얼셀
한화솔루션
ecopro
shinsungeng
unison
kakao
naver
nc_s
sds
samsung_e
sk_h
skc
hansol
DB
lotte_shoping
shinse
samsungcnt
dongsuh
kb
korea
shinhan
woori
hana


## 40페이지 종목별로 긁어와서 엑셀 파일 저장

In [7]:
writer = pd.ExcelWriter('naver_cr_2.xlsx', engine='xlsxwriter')
title=[]
for k,v in acode.items():
    a = naver_crawler(v,40)
    title.append(a)
    
    

================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 10 i

In [26]:
# 리스트 형식으로 종목 저장
list_name=[]
for k,v in acode.items():
    list_name.append(k)
print(list_name)

['호텔신라', '두산퓨얼셀', '한화솔루션', 'ecopro', 'shinsungeng', 'unison', 'kakao', 'naver', 'nc_s', 'sds', 'samsung_e', 'sk_h', 'skc', 'hansol', 'DB', 'lotte_shoping', 'shinse', 'samsungcnt', 'dongsuh', 'kb', 'korea', 'shinhan', 'woori', 'hana']


In [27]:
list_name[0]

'호텔신라'

In [30]:
# 엑셀 파일에 자료 쓰기 (보완 사항 : 몽고 디비)
for i in range(0,24):
    title[i].to_excel(writer, sheet_name= list_name[i])
writer.save()

In [33]:
'''
# 1. 엑셀 파일 열기 w/ExcelWriter
writer = pd.ExcelWriter('naver_cr.xlsx', engine='xlsxwriter')

# 2. 시트별 데이터 추가하기(1개 시트로 저장할때와 유사)
doosanfuelcell.to_excel(writer, sheet_name= '두산퓨얼셀')
hanwhasolutions.to_excel(writer, sheet_name= '한화솔류션')
ecopro.to_excel(writer, sheet_name= '에코프로')
shinsungeng.to_excel(writer, sheet_name= '신성이엔지')
unison.to_excel(writer, sheet_name= '유니슨')
kakao.to_excel(writer, sheet_name= '카카오')
naver.to_excel(writer, sheet_name= '네이버')
nc_s.to_excel(writer, sheet_name= '엔씨소프트')
sds.to_excel(writer, sheet_name= 'SDS')
samsung_e.to_excel(writer, sheet_name= '삼성전자')
sk_h.to_excel(writer, sheet_name= 'SK하이닉스')
skc.to_excel(writer, sheet_name= 'SKC')
hansol.to_excel(writer, sheet_name= '한솔케미칼 ')
DB.to_excel(writer, sheet_name= 'DB하이텍')
lotte_shoping.to_excel(writer, sheet_name= '롯데쇼핑')
shinse.to_excel(writer, sheet_name= '신세계')
data_hotel.to_excel(writer, sheet_name= '신라호텔')
samsungcnt.to_excel(writer, sheet_name= '삼성물산')
dongsuh.to_excel(writer, sheet_name= '동서')
kb.to_excel(writer, sheet_name= 'KB금융')
korea.to_excel(writer, sheet_name= '한국금융')
shinhan.to_excel(writer, sheet_name= '신한지주')
woori.to_excel(writer, sheet_name= '우리금융지주')
hana.to_excel(writer, sheet_name= '하나금융지주')


# 3. 엑셀 파일 저장하기
writer.save()
'''


"\n# 1. 엑셀 파일 열기 w/ExcelWriter\nwriter = pd.ExcelWriter('naver_cr.xlsx', engine='xlsxwriter')\n\n# 2. 시트별 데이터 추가하기(1개 시트로 저장할때와 유사)\ndoosanfuelcell.to_excel(writer, sheet_name= '두산퓨얼셀')\nhanwhasolutions.to_excel(writer, sheet_name= '한화솔류션')\necopro.to_excel(writer, sheet_name= '에코프로')\nshinsungeng.to_excel(writer, sheet_name= '신성이엔지')\nunison.to_excel(writer, sheet_name= '유니슨')\nkakao.to_excel(writer, sheet_name= '카카오')\nnaver.to_excel(writer, sheet_name= '네이버')\nnc_s.to_excel(writer, sheet_name= '엔씨소프트')\nsds.to_excel(writer, sheet_name= 'SDS')\nsamsung_e.to_excel(writer, sheet_name= '삼성전자')\nsk_h.to_excel(writer, sheet_name= 'SK하이닉스')\nskc.to_excel(writer, sheet_name= 'SKC')\nhansol.to_excel(writer, sheet_name= '한솔케미칼 ')\nDB.to_excel(writer, sheet_name= 'DB하이텍')\nlotte_shoping.to_excel(writer, sheet_name= '롯데쇼핑')\nshinse.to_excel(writer, sheet_name= '신세계')\ndata_hotel.to_excel(writer, sheet_name= '신라호텔')\nsamsungcnt.to_excel(writer, sheet_name= '삼성물산')\ndongsuh.to_excel(writer, sheet